In [62]:
#%matplotlib inline
import os
import numpy as np
from PIL import Image
import matplotlib as mp
from matplotlib import pyplot,pylab
plt = pyplot
import scipy
from __future__ import division
import seaborn as sns
sns.set_style('white')
import string
import pandas as pd
from scipy import stats

import json
import pymongo as pm

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")

## purpose

* upload sketches to S3 [maybe do this later]
* build stimulus dictionary and write to database

## upload sketches to S3 [todo later]

## build stimulus dictionary

In [57]:
stroke_count = [d['svg'].count('M') for i,d in meta.iterrows()]

[7,
 11,
 14,
 5,
 14,
 16,
 8,
 6,
 7,
 6,
 6,
 4,
 5,
 5,
 11,
 8,
 23,
 11,
 20,
 8,
 10,
 16,
 17,
 20,
 7,
 12,
 15,
 7,
 7,
 5,
 9,
 8,
 5,
 4,
 8,
 8,
 10,
 6,
 4,
 15,
 12,
 13,
 9,
 5,
 11,
 11,
 9,
 4,
 4,
 8,
 15,
 5,
 5,
 12,
 5,
 8,
 9,
 10,
 15,
 13,
 9,
 12,
 9,
 16,
 9,
 14,
 11,
 7,
 7,
 8,
 7,
 8,
 14,
 15,
 8,
 24,
 19,
 14,
 9,
 20,
 10,
 6,
 17,
 7,
 18,
 11,
 11,
 7,
 8,
 3,
 15,
 4,
 7,
 9,
 3,
 2,
 4,
 6,
 10,
 25,
 10,
 17,
 25,
 11,
 2,
 17,
 2,
 2,
 6,
 3,
 17,
 6,
 83,
 14,
 23,
 35,
 9,
 12,
 9,
 9,
 8,
 11,
 6,
 11,
 6,
 4,
 5,
 9,
 8,
 8,
 12,
 8,
 6,
 4,
 16,
 9,
 21,
 6,
 4,
 9,
 6,
 6,
 6,
 2,
 17,
 16,
 26,
 9,
 27,
 9,
 8,
 12,
 8,
 8,
 10,
 9,
 7,
 5,
 4,
 10,
 11,
 7,
 13,
 9,
 12,
 7,
 27,
 21,
 5,
 9,
 10,
 4,
 10,
 6,
 5,
 6,
 7,
 12,
 8,
 5,
 1,
 5,
 7,
 13,
 6,
 17,
 19,
 8,
 5,
 6,
 10,
 6,
 11,
 16,
 19,
 10,
 13,
 14,
 10,
 13,
 4,
 7,
 6,
 9,
 16,
 13,
 7,
 6,
 13,
 10,
 5,
 11,
 11,
 11,
 9,
 5,
 16,
 9,
 8,
 7,
 9,
 13,
 10,
 20,
 18,
 8

In [54]:
d['svg'].count('M')

23

In [63]:
## read in experimental metadata file (CSV)
path_to_metadata = 'sketchpad_basic_allcats.csv'
_meta = pd.read_csv(path_to_metadata)   ### raw meta with all categories

## subset by chairs
meta_chairs = _meta[_meta['category']=='chair'] ### subsetted meta with just chairs
meta_chairs = meta_chairs[meta_chairs['numStrokes']<20]
meta_chairs.reset_index(inplace=True)

## assign which meta we will actually upload to mongo in this session
category_flag = 'chairs' ## options: ['allcats','chairs']

if category_flag == 'chairs':
    meta = meta_chairs
elif category_flag == 'allcats':
    meta = _meta
meta

,index,gameID,trialNum,condition,target,drawDuration,outcome,response,numStrokes,Distractor1,Distractor2,Distractor3,pose,iteration,svg,viewerRT,mean_intensity,category
0,0,7004-6ff0964c-ff95-40f9-8696-d6a8931c94d5,1,further,knob,18.389,True,knob,7,crow,redsport,basset,35,pilot2,"[u'M69,167c0,27.13435 5,54.84972 5,81c0,1.8150...",6.656,0.0347,chair
1,8,7004-6ff0964c-ff95-40f9-8696-d6a8931c94d5,9,closer,waiting,21.246,True,waiting,11,leather,inlay,squat,35,pilot2,"[u'M65,288c0,-47.26597 -10.61179,-88.33968 29,...",1.793,0.0722,chair
2,9,7004-6ff0964c-ff95-40f9-8696-d6a8931c94d5,10,closer,inlay,25.374,True,inlay,14,leather,waiting,squat,35,pilot2,"[u'M53,198c5.4241,-32.54462 25.86878,-21.08688...",1.847,0.0566,chair
3,16,7004-6ff0964c-ff95-40f9-8696-d6a8931c94d5,17,further,straight,4.712,True,straight,5,nightingale,white,goldenretriever,35,pilot2,"[u'M49,166c0,7.67391 -0.75258,15.33008 -1,23c-...",31.134,0.0243,chair
4,22,7004-6ff0964c-ff95-40f9-8696-d6a8931c94d5,23,closer,leather,37.167,True,leather,14,inlay,waiting,squat,35,pilot2,"[u'M49,280c-2.11228,0 -0.86365,-5.63654 -1,-7c...",6.244,0.0663,chair
5,24,7004-6ff0964c-ff95-40f9-8696-d6a8931c94d5,25,closer,squat,33.205,True,squat,16,leather,inlay,waiting,35,pilot2,"[u'M41,93c0,38.48471 -11.67573,78.26991 -6,118...",1.505,0.0542,chair
6,25,7004-6ff0964c-ff95-40f9-8696-d6a8931c94d5,26,further,woven,12.926,True,woven,8,tomtit,brown,bloodhound,35,pilot2,"[u'M59,258c0,-22.55708 8,-45.20954 8,-68', u'M...",4.021,0.0421,chair
7,28,7004-6ff0964c-ff95-40f9-8696-d6a8931c94d5,29,further,sling,7.536,True,sling,6,robin,redantique,bullmastiff,35,pilot2,"[u'M75,184c-21.34375,0 -1.69512,42.65843 -2,64...",2.633,0.0394,chair
8,32,6224-40ab6004-b7c1-4d1d-a308-a061099290fc,1,closer,straight,13.853,True,straight,7,squat,inlay,waiting,35,pilot2,"[u'M87,28c0,32.40293 11.48971,63.4074 15,95c2....",8.662,0.0488,chair
9,35,6224-40ab6004-b7c1-4d1d-a308-a061099290fc,4,further,knob,10.920,True,knob,6,pigeon,redsport,chihuahua,35,pilot2,"[u'M92,139c-13.86192,13.86192 -5.0107,49.53744...",3.621,0.0270,chair


In [64]:
## add parts list
parts =[]
for i in range(meta.shape[0]-1):
    if meta.category[i]=="chair":
        parts.append(["backrest,armrest,seat,leg"])        
    if meta.category[i]=="dog":
        parts.append(["eye,mouth,ear,head,neck,body,leg,tail"])
    if meta.category[i] == "bird":
        parts.append(["beak,head,body,wings,feet,tail"])
    if meta.category[i] == "car":
        parts.append(["bumper,lights,windshield,body,window,door,wheel"])
meta = meta.assign(parts=pd.Series(parts))


In [ ]:
## add iteration name information
_iterationName = 'sketchpad_basic_{}'.format(category_flag)
iterationName = [_iterationName]*len(meta)
meta = meta.assign(iterationName=pd.Series(iterationName))

In [ ]:
## svg string formatting
svg = []
for i,d in meta.iterrows():    
    splitted = d['svg'].split("'") ## parse string to re-split up into strokes
    svgString = [i for i in splitted if i[0]=='M'] ## check to make sure it is a real start of a spline
    svg.append(svgString)
meta = meta.assign(svg=pd.Series(svg)) 

In [ ]:
## add empty games list
games = [[] for i in np.arange(len(meta))]
meta = meta.assign(games=pd.Series(games))

In [ ]:
## how many games worth of data do we have?
print '{} unique games worth of data.'.format(len(np.unique(meta.gameID.values)))
print '{} unique sketches.'.format(len(meta))

## write out metadata to json file

## for example:
stimdict = meta.to_dict(orient='records')
stimdict
import json
with open('annotation_meta.js', 'w') as fout:
     json.dump(stimdict, fout)

### upload stim dictionary to mongo (db = 'stimuli', collection='sketchpad_basic_recog')

In [ ]:
## load in the JSON that contains the svgData, object labels, and part labels
J = json.loads(open('annotation_meta.js',mode='ru').read())
assert len(J)==len(meta)

In [ ]:
# set vars 
auth = pd.read_csv('auth.txt', header = None) # this auth.txt file contains the password for the sketchloop user
pswd = auth.values[0][0]
user = 'sketchloop'
host = 'rxdhawkins.me' ## cocolab ip address

# have to fix this to be able to analyze from local
conn = pm.MongoClient('mongodb://sketchloop:' + pswd + '@127.0.0.1')

In [ ]:
## define the dbname and collection name
db = conn['stimuli']
coll = db['svg_annotation_sketchpad_basic_{}'.format(category_flag)]

In [ ]:
## actually add data now to the database (iff reallyRun==True)
reallyRun = True
if reallyRun:
    for (i,j) in enumerate(J):
        if i%250==0:
            print ('%d of %d' % (i,len(J)))
        coll.insert_one(j)
reallyRun = False        

In [ ]:
## How many sketches do we have in the database?
print 'We have {} sketches.'.format(coll.count())

## What kind of sketches do we have in the database?
print 'We have these kinds: {}'.format(str(coll.distinct('category')))